# Odd Network

**TLDR;**
Given a network (represented by two arrays), find the number of paths with odd number of links.


You analyze the performance of a computer network. The network comprises nodes connected by peer-to-peer links. There are N links and N + 1 nodes. All pairs of nodes are (directly or indirectly) connected by links, and links don't form cycles. In other words, the network has a tree topology.

Your analysis shows that communication between two nodes performs much better if the number of links on the (shortest) route between the nodes is odd. Of course, the communication is fastest when the two nodes are connected by a direct link. But, amazingly, if the nodes communicate via 3, 5, 7, etc. links, communication is much faster than if the number of links to pass is even.

Now you wonder how this influences the overall network performance. There are N * (N + 1) / 2 different pairs of nodes. You need to compute how many of them are pairs of nodes connected via an odd number of links.

Nodes are numbered from 0 to N. Links are described by two arrays of integers, A and B, each containing N integers. For each 0 ≤ I < N, there is a link between nodes A[I] and B[I].

Write a function:

```
def solution(A, B)
```

that, given two arrays, A and B, consisting of N integers and describing the links, computes the number of pairs of nodes X and Y, such that 0 ≤ X < Y ≤ N, and X and Y are connected via an odd number of links.

EXAMPLE 1

For example, given N = 6 and the following arrays:

```
  A[0] = 0    B[0] = 3
  A[1] = 3    B[1] = 1
  A[2] = 4    B[2] = 3
  A[3] = 2    B[3] = 3
  A[4] = 6    B[4] = 3
  A[5] = 3    B[5] = 5
  
  0  1  2 
   \ | /
     3
   / | \
  4  5  6
```

the function should return 6, since:

  - there are six pairs of nodes connected by direct links:
    [[0, 3], [1, 3], [2, 3], [3, 4], [3, 5], [3, 6]]
  - all other pairs of nodes are connected via two links.


EXAMPLE 2

Given N = 5 and the following arrays:

```
  A[0] = 0    B[0] = 1
  A[1] = 4    B[1] = 3
  A[2] = 2    B[2] = 1
  A[3] = 2    B[3] = 3
  A[4] = 4    B[4] = 5
  
  0 - 1 - 2 - 3 - 4 - 5
```

the function should return 9, since:

  - solution: [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [0, 1, 2, 3], [1, 2, 3, 4], [2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]
  - there are five pairs of nodes connected by direct links,
  - there are three pairs of nodes connected via three links, and
  - there is one pair of nodes connected via five links.

EXAMPLE 3

Given N = 7 and the following arrays:

```
  A[0] = 0    B[0] = 3
  A[1] = 4    B[1] = 5
  A[2] = 4    B[2] = 1
  A[3] = 2    B[3] = 3
  A[4] = 7    B[4] = 4
  A[5] = 6    B[5] = 3
  A[6] = 3    B[6] = 4
  
      0   1
      |   |
  2 - 3 - 4 - 5
      |   |
      6   7
```

the function should return 16, since:
  - solution: [[0, 3], [1, 4], [2, 3], [3, 4], [3, 6], [4, 5], [4, 7],
    [0, 3, 4, 1], [0, 3, 4, 5], [0, 3, 4, 7], [1, 4, 3, 2], [1, 4, 3, 6], [2, 3, 4, 5], [2, 3, 4, 7], [5, 4, 3, 6], [6, 3, 4, 7]]
  - there are seven pairs of nodes connected by direct links, and
  - there are nine pairs of nodes connected via three links.

Write an efficient algorithm for the following assumptions:

  - N is an integer within the range [0..90,000];
  - each element of arrays A and B is an integer within the range [0..N];
  - the network has a tree topology;
  - any pair of nodes is connected via no more than 1000 links.

Ref:
- Code challenging from Opus, 2022-10-18.
- https://app.codility.com/cert/view/cert92A85Y-4VM22MZ3A3EC3YE9/details/
- Level: medium (but tedious)


In [1]:
from typing import List, Set
from collections import defaultdict
from itertools import combinations


class Solution:
    @classmethod
    def create_network(cls, A: List[int], B: List[int]) -> List[Set[int]]:
        network = defaultdict(set)
        for a, b in zip(A,B):
            network[a].add(b)
            network[b].add(a)
        return network

    def solution_1(self, A: List[int], B: List[int]) -> int:
        """BFS, with queues.
        """
        network = self.create_network(A, B)
        # print(f"[DEBUG] Network: {network}")

        odd_link_paths = []
        queue = [[n] for n in sorted(network.keys())]
        # print(f"[DEBUG] Initial queue = {queue}")
        while queue:
            # print(f"[DEBUG] ## Queue = {queue}")
            next_queue = []
            for path in queue:
                # print(f"[DEBUG] checking path: {path} ...")
                n_start = path[0]
                n_end = path[-1]
                curr_num_links = len(path) - 1
                for c in network[n_end]:
                    # Make sure that the child node is not part of the path
                    if c not in path:
                        # Build a new path; note that we need to copy the current path.
                        new_path =(*path, c)
                        # Keep the odd-link paths where the start node is less than the end node,
                        # so as to avoid duplications.
                        if curr_num_links % 2 == 0 and (n_start < c):
                            odd_link_paths.append(new_path)
                        next_queue.append(new_path)
            queue = next_queue
        print(f"[DEBUG] odd_link_paths = {odd_link_paths}")
        return len(odd_link_paths)

    def solution_2(self, A: List[int], B: List[int]) -> int:
        """DFS, with recursion.
        The logic is about the same as v1.
        Recursion makes the code simplier.
        Also, we don't need to copy the path until adding them to the results.
        """
        def dfs(path: List[int], network: dict, odd_link_paths: list):
            # Check if the path is has an odd number of links
            n_start = path[0]
            n_end = path[-1]
            num_links = len(path) - 1
            if (num_links > 0) and (num_links % 2 == 1) and (n_start < n_end):
                odd_link_paths.append(tuple(path))

            # Expand the paths
            for c in network[n_end]:
                if c not in path:
                    path.append(c)  # path is modified here
                    dfs(path, network, odd_link_paths)
                    path.pop()  # restore back to the original path

        network = self.create_network(A, B)
        odd_link_paths = []
        for k in sorted(network.keys()):
            dfs([k], network, odd_link_paths)

        print(f"[DEBUG] odd_link_paths = {odd_link_paths}")
        return len(odd_link_paths)

    def solution_3(self, A: List[int], B: List[int]) -> int:
        """Treat it as a tree and apply DFS."""

        def dfs(curr_node: int, network: dict, seen: set, odd_link_paths: list) -> List[list]:
            seen.add(curr_node)

            # Direct paths
            child_paths_list = []
            return_paths = []
            return_paths.append([curr_node])
            for c in network[curr_node]:
                if c in seen:
                    continue
                child_paths = dfs(c, network, seen, odd_link_paths)
                for child_path in child_paths:
                    path = (curr_node, *child_path)
                    if len(path) % 2 == 0:
                        odd_link_paths.append(path)
                    return_paths.append(path)
                child_paths_list.append(child_paths)

            # Cross paths: child1_path + curr_node + child2_path
            # Find all possible combinations
            for path_list1, path_list2 in combinations(child_paths_list, 2):
                for p1 in path_list1:
                    n1 = len(p1)
                    for p2 in path_list2:
                        n2 = len(p2)
                        if (n1 + n2) % 2 == 1:
                            path = (*p1, curr_node, *p2)
                            odd_link_paths.append(path)

            return return_paths

        network = self.create_network(A, B)
        node = A[0]
        seen = set()
        odd_link_paths = []
        dfs(node, network, seen, odd_link_paths)
        # odd_link_paths = [p if p[0] < p[-1] else list(reversed(p)) for p in odd_link_paths]
        print(f"[DEBUG] odd_link_paths = {odd_link_paths}")
        return len(odd_link_paths)

    def solution_4(self, A: List[int], B: List[int]) -> int:
        """Breadth First.  
        Start from every node. Exhaust all combinations.

        Time complexity: O(N^3).
        """
    
        # Build the network: {node: [node]}
        network = self.create_network(A, B)
            
        # Use the breath-first search (BFS) method to explore the network
        # Use the queue to list the nodes that need to be processed next.
        # Each element is (start_node, current_node).
        queue = [(k,k) for k in network.keys()] 

        # Use a dictionary to track all of the links that have been processed.
        # (n1, n2) and (n2, n1) are considered different.
        seen = dict()   # { (begin, end) : num_links }
        num_links = 0
        while queue:
            next_queue = []
            num_links += 1
            for n_start, n_curr in queue:
                # Find all nodes that are connected to the "current" node.
                connected_nodes = network[n_curr]
                for n_next in connected_nodes:
                    node_pair = (n_start, n_next)
                    if node_pair not in seen:
                        seen[node_pair] = num_links
                        next_queue.append([n_start, n_next])
            queue = next_queue

        # From all of the identified links, find out those with odd number of links
        odd_paths = dict()
        for (n1,n2), v in seen.items():
            if v %2 == 1:
                # Make sure n1 < n2, to avoid duplications.
                if (n2 < n1):
                    n1, n2 = n2, n1 
                odd_paths[(n1, n2)] = v

        # Return the number of odd links
        return len(odd_paths)

    def solution_5(self, A: List[int], B: List[int]) -> int:
        """ DFS - Depth-First Search. 
        This solution doesn't track the complete paths.

        Need to primarily handle two situations:
        1) directy -- links with the parents
        2) cross -- links among children only.
        """       
        
        def dfs(n: int, network: dict, seen: set, odd_links: List[int]) -> List[int]:
            """Get "direct" links from children.
            Find out all posible inter-children links.
            Then return direct links (including self) back to the parent.
            
            :param n: the current node value
            :param network: dictionary of {node: set}
            :param odd_links: the results
            :return: a list of number of links from the children nodes.
            """
            seen.add(n)
            children = network[n]      # a set of node values connected to node 'n'
            children_links_list = []   # a list of lists
            return_links = []          # a list of integers
            
            # Handle direct links.
            # These links will be passed to the parent nodes.
            for c in children:
                # Make sure that the child has not been processed yet
                if c not in seen:
                    # Get a list of integers, representing the number of links
                    links = dfs(c, network, seen, odd_links)
                    for x in links:
                        # Check if the number is odd
                        if x % 2 == 1:
                            odd_links.append(x)

                        # Add one (the current node)
                        return_links.append(x + 1)
                    children_links_list.append(links)
                    
            # Handle cross links (process two links pairs at a time)
            # Note that children_links_list is a list of lists
            # Use "combinations" to find out all of the children pairs.
            for p in combinations(children_links_list, 2):
                list1 = p[0]
                list2 = p[1]
                # Exhaust all combinations between two lists of links
                for x in list1:
                    for y in list2:
                        z = x + y
                        if z % 2 == 1:
                            odd_links.append(z)
                               
            # Add self to the reslt links
            return_links.append(1)   
      
            return return_links
            
        # Build the network: {node: {node}}
        network = self.create_network(A, B)
            
        odd_links = []
        seen = set()
        n = A[0]   # Can start with any node.
        dfs(n, network, seen, odd_links)
        return len(odd_links)


def main():
    """Main function"""
    test_data = [
        [[0, 3, 4, 2, 6, 3], [3, 1, 3, 3, 3, 5], 6],
        [[0, 4, 2, 2, 4], [1, 3, 1, 3, 5], 9],
        [[0, 4, 4, 2, 7, 6, 3], [3, 5, 1, 3, 4, 3, 4], 16],
    ]
    ob1 = Solution()
    for A, B, ans in test_data:
        print(f"\n# Input: A={A}, B={B} (ans={ans})")
        print(f"  Output 1 = {ob1.solution_1(A, B)}")
        print(f"  Output 2 = {ob1.solution_1(A, B)}")
        print(f"  Output 3 = {ob1.solution_3(A, B)}")
        print(f"  Output 4 = {ob1.solution_4(A, B)}")
        print(f"  Output 5 = {ob1.solution_5(A, B)}")
        
main()


# Input: A=[0, 3, 4, 2, 6, 3], B=[3, 1, 3, 3, 3, 5] (ans=6)
[DEBUG] odd_link_paths = [(0, 3), (1, 3), (2, 3), (3, 4), (3, 5), (3, 6)]
  Output 1 = 6
[DEBUG] odd_link_paths = [(0, 3), (1, 3), (2, 3), (3, 4), (3, 5), (3, 6)]
  Output 2 = 6
[DEBUG] odd_link_paths = [(3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (0, 3)]
  Output 3 = 6
  Output 4 = 6
  Output 5 = 6

# Input: A=[0, 4, 2, 2, 4], B=[1, 3, 1, 3, 5] (ans=9)
[DEBUG] odd_link_paths = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (0, 1, 2, 3), (1, 2, 3, 4), (2, 3, 4, 5), (0, 1, 2, 3, 4, 5)]
  Output 1 = 9
[DEBUG] odd_link_paths = [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (0, 1, 2, 3), (1, 2, 3, 4), (2, 3, 4, 5), (0, 1, 2, 3, 4, 5)]
  Output 2 = 9
[DEBUG] odd_link_paths = [(4, 5), (3, 4), (2, 3), (2, 3, 4, 5), (1, 2), (1, 2, 3, 4), (0, 1), (0, 1, 2, 3), (0, 1, 2, 3, 4, 5)]
  Output 3 = 9
  Output 4 = 9
  Output 5 = 9

# Input: A=[0, 4, 4, 2, 7, 6, 3], B=[3, 5, 1, 3, 4, 3, 4] (ans=16)
[DEBUG] odd_link_paths = [(0, 3), (1, 4), (2, 3), (3, 4), (3